In [1]:
import os
import json
import math
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from icecream import ic
ROOT = "/mnt/Data-Work-RE/26_Agricultural_Engineering-RE/263_DP/00_Darwin/digital-production"

In [2]:
def get_highest_release_dir(root, dataset_name):
    releases_path = os.path.join(root, dataset_name, "releases")
    try:
        dirs = [d for d in os.listdir(releases_path) if os.path.isdir(os.path.join(releases_path, d))]
        numbered_dirs = [int(d) for d in dirs if d.isdigit()]
        if not numbered_dirs:
            return None
        return str(max(numbered_dirs))
    except FileNotFoundError:
        print(f"Path not found: {releases_path}")
        return None

# Example usage:
dataset_name = "lightly"
release_id = get_highest_release_dir(ROOT, dataset_name)
release_id

'2'

In [7]:
def process_bounding_boxes(x, y, w, h, j, i, tile_width, tile_height):
    # Calculate the overlapping region
    x1 = max(x, j)
    y1 = max(y, i)
    x2 = min(x + w, j + tile_width)
    y2 = min(y + h, i + tile_height)
    # print(f'From pbb function: i = {i}')
    # print(f'From pbb function: j = {j}')
    # print(f'From pbb function: x1 = {x1}')
    # print(f'From pbb function: y1 = {y1}')
    # print(f'From pbb function: x2 = {x2}')
    # print(f'From pbb function: y2 = {y2}')

    # Check if there is an overlap
    if x1 < x2 and y1 < y2:
        # print(f'From pbb function: x1 - j = {x1 - j}')
        # print(f'From pbb function: y1 - i = {y1 - i}')
        # print(f'From pbb function: x2 - x1 = {x2 - x1}')
        # print(f'From pbb function:  y2 - y1 = { y2 - y1}')

        return (x1 - j, y1 - i, x2 - x1, y2 - y1) # newX, newY, newWidth, newHeight
    else:
        return None

def tile_and_save(image_path, annotation_path, out_image_dir, out_ann_dir, target_class='rumex_plant'):
    # Create output directories if they don't exist
    os.makedirs(out_image_dir, exist_ok=True)
    os.makedirs(out_ann_dir, exist_ok=True)

    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image: {image_path}")
        return

    image_height, image_width = image.shape[:2]

    # Load the annotation file
    with open(annotation_path, 'r') as f:
        annotation_data = json.load(f)

    tile_count = 0
    
    # Tile the image
    tile_width, tile_height = 1024, 678
    for i in range(0, image_height - tile_height + 1, tile_height):
        remaining_height = image_height - i
        if remaining_height < 50:
            break  # Stop if remaining height is less than 10 pixels

        for j in range(0, image_width - tile_width + 1, tile_width):
            remaining_width = image_width - j
            if remaining_width < 50:
                break  # Stop if remaining width is less than 10 pixels

            tile = image[i:i + tile_height, j:j + tile_width]
            tile_bboxes = []

            # Check if the tile contains any bounding boxes
            for annotation in annotation_data['annotations']:
                if annotation['name'] == target_class:
                    bbox = annotation['bounding_box']
                    x, y, w, h = bbox['x'], bbox['y'], bbox['w'], bbox['h']
                    # print(f'From loop: x1 = {x}')
                    # print(f'From loop: x1 = {y}')
                    # print(f'From loop: x1 = {w}')
                    # print(f'From loop: x1 = {h}')

                    adjusted_bbox = process_bounding_boxes(x, y, w, h, j, i, tile_width, tile_height)
                    # print(adjusted_bbox)
                    if adjusted_bbox:
                        tile_bboxes.append(adjusted_bbox)

            if tile_bboxes:
                # Save the tile
                tile_image_path = os.path.join(out_image_dir, f"{os.path.splitext(os.path.basename(image_path))[0]}_{i}_{j}.png")
                cv2.imwrite(tile_image_path, tile)

                # Create and save the annotation for the tile
                tile_annotation = {
                    "version": "2.0",
                    "schema_ref": "https://darwin-public.s3.eu-west-1.amazonaws.com/darwin_json/2.0/schema.json",
                    "item": {
                        "name": os.path.basename(tile_image_path),
                        "path": "/",
                        "slots": [{
                            "type": "image",
                            "slot_name": "0",
                            "width": tile_width,
                            "height": tile_height,
                            "thumbnail_url": "",
                            "source_files": [{"file_name": os.path.basename(tile_image_path), "url": "", "local_path": tile_image_path}]
                        }]
                    },
                    "annotations": [
                        {
                            "bounding_box": {"h": bbox[3], "w": bbox[2], "x": bbox[0], "y": bbox[1]},
                            "name": target_class,
                            "properties": [],
                            "slot_names": ["0"]
                        } for bbox in tile_bboxes
                    ],
                    "properties": []
                }

                tile_annotation_path = os.path.join(out_ann_dir, f"{os.path.splitext(os.path.basename(annotation_path))[0]}_{i}_{j}.json")
                with open(tile_annotation_path, 'w') as f:
                    json.dump(tile_annotation, f, indent=4)
                tile_count += 1

    return tile_count

def process_dataset(ROOT, dataset_name, max_tiles = 10000000):
    total_tiles_saved = 0
    
    img_dir = os.path.join(ROOT, dataset_name, "images")
    ann_dir = os.path.join(ROOT, dataset_name, "releases", get_highest_release_dir(ROOT, dataset_name), "annotations")
    out_img_dir = os.path.join(ROOT, dataset_name, "images_splitted")
    out_ann_dir = os.path.join(ROOT, dataset_name, "releases", get_highest_release_dir(ROOT, dataset_name), "annotations_splitted")

    # Process each image and annotation file
    for image_file in tqdm(os.listdir(img_dir)):
        if image_file.endswith(('.png', 'jpg', '.JPG', '.jpeg')):
            image_path = os.path.join(img_dir, image_file)
            annotation_file = os.path.splitext(image_file)[0] + ".json"
            annotation_path = os.path.join(ann_dir, annotation_file)

            if os.path.exists(annotation_path):
                tiles_saved = tile_and_save(image_path, annotation_path, out_img_dir, out_ann_dir)
                total_tiles_saved += tiles_saved
                if total_tiles_saved >= max_tiles:
                    print(f"Reached the limit of {max_tiles} tiles.")
                    return
            else:
                print(f"Annotation file not found for image: {image_file}")

# Example usage
dataset_name = "lightly"
process_dataset(ROOT, dataset_name)

  3%|▎         | 23/785 [00:20<13:02,  1.03s/it]

Annotation file not found for image: 20230621_HerrenpuentSuedwestStreifen_S_00_F_00_H_12_O_sama_ID1_DJI_20230621113335_0024.2_2.png


100%|██████████| 785/785 [14:07<00:00,  1.08s/it]


In [8]:
# Example usage
dataset_name = "bildacher"
process_dataset(ROOT, dataset_name)

100%|██████████| 220/220 [09:02<00:00,  2.47s/it]


In [9]:
dataset_name = "haldennord10"
process_dataset(ROOT, dataset_name)

100%|██████████| 160/160 [17:25<00:00,  6.54s/it]


# The below codes are for debugging purposes only

In [ ]:
im_name = '20230621_HerrenpuentSuedwestStreifen_S_00_F_00_H_12_O_sama_ID1_DJI_20230621113335_0024.2_2.PNG'
json_name =  im_name.replace('PNG', 'json')
dataset = 'lightly'
im_path = os.path.join(ROOT, dataset, 'images', im_name)
json_path = os.path.join(ROOT, f'{dataset}/releases/1/annotations', json_name)

image = cv2.imread(im_path)
if image is None:
    raise FileNotFoundError(f"Could not load image at: {im_name}")
image_height, image_width = image.shape[:2]

# === Draw bounding boxes (in blue) ===
with open(json_path, 'r') as f:
    data = json.load(f)

annotations = data.get("annotations", [])
for ann in annotations:
    bbox = ann.get("bounding_box", {})
    x, y = int(bbox["x"]), int(bbox["y"])
    w, h = int(bbox["w"]), int(bbox["h"])
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Blue boxes

# === Draw tile grid lines (in red) ===
# for i in range(0, image_height - tile_height + 1, tile_height):
#     cv2.line(image, (0, i), (image_width, i), color=(0, 0, 255), thickness=10)  # Horizontal

# for j in range(0, image_width - tile_width + 1, tile_width):
#     cv2.line(image, (j, 0), (j, image_height), color=(0, 0, 255), thickness=10)  # Vertical

# === Save image ===
cv2.imwrite(f'boxes_{im_name}', image)

# === Display using matplotlib ===
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(12, 8))
plt.imshow(image_rgb)
plt.axis("off")
plt.title("Annotations + Grid")
plt.show()

In [ ]:

im_name = os.path.join(ROOT, 'lightly', 'images', '20230621_HerrenpuentSuedwestStreifen_S_00_F_00_H_12_O_sama_ID1_DJI_20230621113335_0024.2_2.png')
json_path = os.path.join(ROOT, 'lightly/releases/1/annotations', '20230621_HerrenpuentSuedwestStreifen_S_00_F_00_H_12_O_sama_ID1_DJI_20230621113335_0024.2_2.json')


# Load image
image = cv2.imread(im_name)
if image is None:
    raise FileNotFoundError(f"Could not load image at: {im_name}")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Load Darwin 2.0 annotation JSON
with open(json_path, 'r') as f:
    data = json.load(f)

annotations = data.get("annotations", [])
print(annotations)

for ann in annotations:
        bbox = ann.get("bounding_box", {})
        print(bbox)
        x, y = int(bbox["x"]), int(bbox["y"])
        w, h = int(bbox["w"]), int(bbox["h"])
        # Draw bounding box
        print(x)
        print(y)
        print(h)
        print(w)

        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)

# Show result
plt.figure(figsize=(12, 8))
plt.imshow(image)
plt.axis("off")
plt.title("Darwin 2.0 Annotations")
plt.show()